# Capítulo 2

In [2]:
%load_ext rpy2.ipython

Error importing in API mode: ImportError('On Windows, cffi mode "ANY" is only "ABI".')
Trying to import in ABI mode.


In [ ]:
%%R
library(rethinking) # McElreath lib
globe.qa <- quap( # quadratic approximation
    alist(
        W ~ dbinom( W+L ,p) , # binomial likelihood
        p ~ dunif(0,1) # uniform prior
    ),
    data=list(W=6,L=3) 
)

# display summary of quadratic approximation
precis( globe.qa )

  mean   sd 5.5% 94.5%
p 0.67 0.16 0.42  0.92


En python no conozco implementado la aproximación cuadrática. Por lo que se integra así:

In [ ]:
import numpy as np
from scipy.optimize import minimize
from scipy.stats import norm

W, L = 6, 3 # Observed data definition.

# 1. log posterior
def logpost(p):
    p = p[0] # gymnastics because we pass an array([0.5]) for the MAP, relevant on interval.
    if p <= 0 or p >= 1: # we set the boundries.
        return -np.inf # if unbounded return -inf.
    return W*np.log(p) + L*np.log(1 - p) # we sum ln(p)

# 2. MAP estimate | MAP is the maximum a posteriori. 
res = minimize(
    lambda x: -logpost(x),
    x0=np.array([0.5]),
    method="L-BFGS-B",
    bounds=[(1e-6, 1 - 1e-6)] # small enough bounds, just numerical safeguards.
)

map_est = res.x[0] # this is the MAP, we now calculate intervals.

#  3. Numerical Hessian
eps = 1e-5 # epsilon
# it is the finite-difference step 
# used to approximate the second derivative

# fpp is "f" prime prime or f''
# This is a scalar because the parameter space is 1-D.
# The Hessian reduces to a 1×1 matrix whose only entry is f''(p).
fpp = ( 
    -logpost([map_est + eps]) # lo
    - 2*-logpost([map_est])   # MAP
    -logpost([map_est - eps]) # hi
) / eps**2

# Hessian = f'' ; covariance = 1/H
var = 1.0 / fpp
sd = np.sqrt(var)

# 4. HDI from Gaussian approx
# Because we're doing a quad approximation
# We asume a normal distribution
# Loc=mu, scale=sd.
lo = norm.ppf(0.055, loc=map_est, scale=sd)
hi = norm.ppf(0.945, loc=map_est, scale=sd)

# This is just done to round the values
map_est, sd, lo, hi = np.array([map_est, sd, lo, hi]).round(2)

print("MAP:", map_est)
print("SD:", sd)
print("89% HDI:", lo, hi)


MAP: 0.67
SD: 0.16
89% HDI: 0.42 0.92


PyMc can be used to calculate the quad map but not directly the confidence interval nor the standard deviation.

In [59]:
import pymc as pm

with pm.Model() as glob_model:
    p = pm.Uniform('p', 0, 1)
    lk = pm.Binomial('likelihood', n=W+L, p=p, observed=W)
    mp = pm.find_MAP()

print('Map approximation:', mp['p'])

Output()

Map approximation: 0.6666666671652423
